In [1]:
import numpy as np
import deepburn.burnup_problem as bup
from deepburn import CRAM
from scipy.integrate import solve_ivp
import scipy.linalg as la
np.set_printoptions(precision=20)

In [2]:
pol = bup.Polonium()

In [3]:
print(pol.time_stamps)

[1728000, 15552000]


In [4]:
print(pol.ref_sols)

{1: array([6.9587617733003087e-04, 7.4518249505036562e-09,
       1.2725788327617256e-08])}


In [5]:
print(pol.time_stamps)

[1728000, 15552000]


In [6]:
print(pol.initial_condition)

[0.000695896 0.          0.         ]


In [7]:
print(pol)
print(pol.densematrix)

Po210
Isotopes: Bi-209 Bi-210 Po-210 
Transition matrix
  (0, 0)	-1.83163e-12
  (1, 0)	1.83163e-12
  (1, 1)	-1.60035e-06
  (2, 1)	1.60035e-06
  (2, 2)	-5.79764e-08
Initial condition
[0.000695896 0.          0.         ]
Time stamps
1728000     No reference solution provided
15552000    [6.9587617733003087e-04 7.4518249505036562e-09 1.2725788327617256e-08]

[[-1.83163e-12  0.00000e+00  0.00000e+00]
 [ 1.83163e-12 -1.60035e-06  0.00000e+00]
 [ 0.00000e+00  1.60035e-06 -5.79764e-08]]


In [8]:
craslit = CRAM.cras_literature
CF4solver = CRAM.CRA_ODEsolver(craslit('VandenEynde2021', 4))
CF5solver = CRAM.CRA_ODEsolver(craslit('VandenEynde2021', 5))
CF16solver = CRAM.CRA_ODEsolver(craslit('VandenEynde2021', 16))
Calvin4solver = CRAM.CRA_ODEsolver(craslit('Calvin2021', 4))

In [9]:
T = 365*24*3600
print(CF4solver._solveCRA(pol.sparsematrix*T, pol.initial_condition))
print(CF5solver._solveCRA(pol.sparsematrix*T, pol.initial_condition))
print(CF16solver._solveCRA(pol.sparsematrix*T, pol.initial_condition))
print(Calvin4solver._solveCRA(pol.sparsematrix*T, pol.initial_condition))

[6.9579575025217107e-04 7.9641937268972325e-10 1.8317104159353888e-08]
[6.958622902158125e-04 7.964269030299222e-10 1.831920304468977e-08]
[6.958558046187419e-04 7.964206743286377e-10 1.831919546848851e-08]
[6.9579571972762139e-04 7.9641931638245576e-10 1.8317103945743355e-08]


In [10]:
#Polonium problem

A = np.array([[-1.83163e-12,0,0],[+1.83163e-12,-1.60035e-6,0],[0,+1.60035e-6,-5.79764e-8]])
def Ydot(t,Y):
    A = np.array([[-1.83163e-12,0,0],[+1.83163e-12,-1.60035e-6,0],[0,+1.60035e-6,-5.79764e-8]])
    return A.dot(Y)
    
Y0 = np.array([6.95896e-4,0,0])
Y_Radau = solve_ivp(Ydot,(0, T), Y0, method="Radau", rtol=1e-13, atol = 1e-30, t_eval=[T])
print(Y_Radau.y)

[[6.9585580461873102e-04]
 [7.9642067432861920e-10]
 [1.8319195468488268e-08]]


In [11]:
la.eig(pol.densematrix*T)

(array([-1.8283437504000000e+00+0.j, -5.0468637600000001e+01+0.j,
        -5.7762283679999996e-05+0.j]),
 array([[ 0.0000000000000000e+00,  0.0000000000000000e+00,
          9.9999999950026375e-01],
        [ 0.0000000000000000e+00,  6.9394413019796986e-01,
          1.1445196959548092e-06],
        [ 1.0000000000000000e+00, -7.2002884953575519e-01,
          3.1593716812716979e-05]]))

In [12]:
lr1 = bup.LagoRahnema_1()
print(lr1)

Lago & Rahnema #1 2017
Isotopes: Th-234 U-238 
Transition matrix
  (0, 1)	4.915959875924379e-18
  (0, 0)	-3.328853448977761e-07
  (1, 1)	-4.915959875924379e-18
Initial condition
[0.e+00 1.e+10]
Time stamps
5e+17    [1.264231274972376e-02 8.560770930100108e+08]



In [13]:
T = lr1.time_stamps[0]
print(CF4solver._solveCRA(lr1.sparsematrix*T, lr1.initial_condition))
print(CF5solver._solveCRA(lr1.sparsematrix*T, lr1.initial_condition))
print(CF16solver._solveCRA(lr1.sparsematrix*T, lr1.initial_condition))
print(Calvin4solver._solveCRA(lr1.sparsematrix*T, lr1.initial_condition))

[1.2616862158184604e-02 8.5521848579158092e+08]
[1.2643971683513569e-02 8.5609599215639591e+08]
[1.2642312749724827e-02 8.5607709301010132e+08]
[1.261685102934048e-02 8.552181708716393e+08]


In [14]:
la.eig(lr1.densematrix*T)

(array([-1.6644267244888806e+11+0.j, -2.4579799379621896e+00+0.j]),
 array([[1.0000000000000000e+00, 1.4767726940657573e-11],
        [0.0000000000000000e+00, 1.0000000000000000e+00]]))

In [15]:
solradau = solve_ivp(lambda t, Y: lr1.densematrix.dot(Y), (0, lr1.time_stamps[0]), [0,1e10], method="Radau", rtol=1e-12, atol=1e-12, t_eval=lr1.time_stamps )

In [16]:
solradau.y

array([[1.2642312749723971e-02],
       [8.5607709301002538e+08]])

In [17]:
prob=lr1
solradau = solve_ivp(lambda t, Y: prob.densematrix.dot(Y), (0, prob.time_stamps[0]), prob.initial_condition, method="Radau", rtol=1e-12, atol=1e-12, t_eval=prob.time_stamps )
solradau.y

array([[1.2642312749723971e-02],
       [8.5607709301002538e+08]])

In [18]:
lr1.densematrix

array([[-3.328853448977761e-07,  4.915959875924379e-18],
       [ 0.000000000000000e+00, -4.915959875924379e-18]])

In [19]:
solradau.status

0

In [20]:
lr2 = bup.LagoRahnema_2()
print(lr2)

Lago & Rahnema #2 2017
Isotopes: Pa-233 U-233 Np-237 
Transition matrix
  (0, 2)	1.0244640277447342e-14
  (0, 0)	-2.974063693062615e-07
  (1, 0)	2.974063693062615e-07
  (1, 1)	-1.3796801963335507e-13
  (2, 2)	-1.0244640277447342e-14
Initial condition
[0.e+00 0.e+00 1.e+12]
Time stamps
1000000000000.0    [3.409551640042892e+04 9.519333617194605e+09 9.898076573074167e+11]



In [21]:
prob=lr2
solradau = solve_ivp(lambda t, Y: prob.densematrix.dot(Y), (0, prob.time_stamps[0]), prob.initial_condition, method="Radau", rtol=1e-12, atol=1e-12, t_eval=prob.time_stamps )
solradau.y

array([[3.409551640042905e+04],
       [9.519333617194639e+09],
       [9.898076573074205e+11]])

In [22]:
lr3 = bup.LagoRahnema_3()
print(lr3)

Lago & Rahnema #3 2017
Isotopes: Tl-207 Pb-207 Pb-211 Bi-211 
Transition matrix
  (0, 3)	0.005398342527725431
  (0, 0)	-0.002421897905520424
  (1, 0)	0.002421897905520424
  (2, 2)	-0.0003200125487349701
  (3, 2)	0.0003200125487349701
  (3, 3)	-0.005398342527725431
Initial condition
[1.e+01 0.e+00 1.e+10 1.e+04]
Time stamps
10000.0    [6.596304013330551e+07 9.500792871334171e+09 4.075708915835030e+08
 2.568320694901933e+07]



In [23]:
prob=lr3
solradau = solve_ivp(lambda t, Y: prob.densematrix.dot(Y), (0, prob.time_stamps[0]), prob.initial_condition, method="Radau", rtol=1e-12, atol=1e-12, t_eval=prob.time_stamps )
solradau.y

array([[6.5963040133305281e+07],
       [9.5007928713341312e+09],
       [4.0757089158350134e+08],
       [2.5683206949019223e+07]])

In [24]:
lr4 = bup.LagoRahnema_4()
print(lr4)

Lago & Rahnema #4 2017
Isotopes: U-235 U-236 U-237 Np-237 
Transition matrix
  (0, 0)	-0.00010000000000003122
  (1, 0)	0.00010000000000003122
  (1, 1)	-0.00010000000000093785
  (2, 1)	0.00010000000000093785
  (2, 2)	-1.1885239721535413e-06
  (3, 2)	1.1885239721535413e-06
  (3, 3)	-1.0244640277447342e-14
Initial condition
[1.e+12 1.e+02 1.e+02 1.e+02]
Time stamps
86400.0    [1.7688690224208966e+08 1.5283028353299613e+09 9.2251944590704846e+11
 7.5775364655379517e+10]



In [25]:
prob=lr4
solradau = solve_ivp(lambda t, Y: prob.densematrix.dot(Y), (0, prob.time_stamps[0]), prob.initial_condition, method="Radau", rtol=1e-12, atol=1e-12, t_eval=prob.time_stamps )
solradau.y

array([[1.7688690224208885e+08],
       [1.5283028353299494e+09],
       [9.2251944590704968e+11],
       [7.5775364628098480e+10]])

In [26]:
lr5 = bup.LagoRahnema_5()
print(lr5)

Lago & Rahnema #5 2017
Isotopes: U-238 U-239 Np-239 Pu-239 Pu-240 Pu-241 Pu-242 Pu-243 Am-241 Am-243 Am-244 Cm-244 
Transition matrix
  (0, 0)	-0.00010000000000000491
  (1, 0)	0.0001
  (1, 1)	-0.0004926419193745169
  (2, 1)	0.0004926419193745169
  (2, 2)	-3.405151448232769e-06
  (3, 2)	3.405151448232769e-06
  (3, 3)	-0.00010000000091101239
  (4, 3)	0.0001
  (4, 4)	-0.00010000000334773796
  (4, 11)	1.2128386927460038e-09
  (5, 4)	0.0001
  (5, 5)	-0.00010000153705449444
  (6, 5)	0.0001
  (6, 6)	-0.00010000153705449444
  (7, 6)	0.0001
  (7, 7)	-3.885005720114481e-05
  (8, 5)	1.5370544944457752e-09
  (8, 8)	-5.07732517929499e-11
  (9, 7)	3.885005720114481e-05
  (9, 9)	-0.00010000005077325179
  (10, 9)	0.0001
  (10, 10)	-1.9063453810779574e-05
  (11, 10)	1.9063453810779574e-05
  (11, 11)	-1.2128386927460038e-09
Initial condition
[1.e+10 1.e+03 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00]
Time stamps
86400.0    [1.7688694812646715e+06 4.5050449123784842e+05 7.76529

In [27]:
prob=lr5
solradau = solve_ivp(lambda t, Y: prob.densematrix.dot(Y), (0, prob.time_stamps[0]), prob.initial_condition, method="Radau", rtol=1e-12, atol=1e-12, t_eval=prob.time_stamps )
solradau.y

array([[1.7688690224249128e+06],
       [4.5050437437825301e+05],
       [7.7652997632872686e+09],
       [2.7308548562202722e+08],
       [2.7995995404871780e+08],
       [2.8211551519812220e+08],
       [2.7585431836631197e+08],
       [5.2964048256264818e+08],
       [2.1477250671970243e+04],
       [1.7237925124463513e+08],
       [3.1221687064040858e+08],
       [1.0719098434701219e+08]])

In [28]:
la.eig(lr5.densematrix*prob.time_stamps[0])

(array([-4.3868089549108714e-06+0.j                 ,
        -3.2775033798060349e-09+0.j                 ,
        -1.6475620144992471e+00+0.j                 ,
        -3.3559228481455188e+00+0.j                 ,
        -8.8637656799754758e+00+0.21358996613631676j,
        -8.8637656799754758e+00-0.21358996613631676j,
        -8.4165430969451158e+00+0.23611509081790985j,
        -8.4165430969451158e+00-0.23611509081790985j,
        -8.6400000787114699e+00+0.j                 ,
        -2.9420508512731125e-01+0.j                 ,
        -4.2564261833958255e+01+0.j                 ,
        -8.6400000000004251e+00+0.j                 ]),
 array([[ 0.0000000000000000e+00+0.0000000000000000e+00j,
          0.0000000000000000e+00+0.0000000000000000e+00j,
          0.0000000000000000e+00+0.0000000000000000e+00j,
          0.0000000000000000e+00+0.0000000000000000e+00j,
          0.0000000000000000e+00+0.0000000000000000e+00j,
          0.0000000000000000e+00-0.0000000000000000e+00j,
  